### Batting Analysis

Uncomment this line to get the most recent statistics from the DakStats website.

In [18]:
#%run ./Conference_Statistics.ipynb

As always, import the necessary statements. Additionally, we change our pandas settings to show all the columns for all of our dataframes, allowing us to view all of the data.

In [2]:
import pandas as pd
import pickle
pd.set_option('display.max_columns', None)

We import our dataframes for batting, pitching, and fielding statistics as well as our list of teams from the pickle file titled `Stats.pkl`.

In [3]:
with open('Stats.pkl', 'rb') as f:
    dfb = pickle.load(f)
    dfp = pickle.load(f)
    dff = pickle.load(f)
    teams = pickle.load(f)

Here, we create a function to calculate some common baseball statistics as well as a few of the more advanced metrics we will be using in our analysis. This function will be useful later on when we gather our league totals so that we do not have to write up all of these same calculations for the new dataframe.

In [4]:
def calcBatStats(df):
    # Batting Average (useful later for League totals)
    df["AVG"] = round(df["H"] / df["AB"], 3)
    # Caught Stealing
    df["CS"] = round(df["SBA"] - df["SB"], 3)
    # Plate Appearances    
    df["PA"] = round(df["AB"] + df["SF"] + df["SH"] + df["BB"] + df["HBP"])
    # OPS
    df["OPS"] = round(df["OBP"] + df["SLG"], 3)
    # Singles
    df["1B"] = df["H"] - df["2B"] - df["3B"] - df["HR"]
    # Runs Created
    df["RC"] = round((df["H"] + df["BB"]) * df["TB"] / (df["AB"] + df["BB"]), 3)
    # POP
    df["POP"] = round(df["OPS"] + df["AVG"], 3)
    # Total Average
    df["TA"] = round((df["TB"] + df["BB"] + df["HBP"] + df["SB"]) / (df["AB"] - df["H"] + df["SH"] + df["SF"] + df["CS"] + df["GDP"]), 3)
    # Batting Average on Balls in Play
    df["BABIP"] = round((df["H"] - df["HR"]) / (df["AB"] - df["SO"] - df["HR"] + df["SF"]), 3)
    # Hoban Efficiency Quotient - Offense
    df["HEQO"] = df["TB"] + df["R"] + df["RBI"] + df["SB"] + (0.5 * df["BB"])
    # Base Stealing Runs
    df["BSR"] = round(((df["H"] + df["BB"] - df["CS"]) * (df["TB"] + (0.7 * df["SB"]))) / (df["AB"] + df["BB"] + df["CS"]), 3)

We run this function on our batting dataframes for each team. Note the newly created columns on the right side of the table. 

In [5]:
for df in dfb:
    calcBatStats(df)
dfb[2][:5]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
1,"Griffin, Chris",28,24.0,0.344,96,17,33,8,0,4,14,53,0.552,15,1,19,0,0.430,2,0,2,2,0,114,0.982,21,22.919,1.326,1.092,0.387,93.5,23.524
2,"Danforth, Patrick",28,24.0,0.333,84,12,28,6,2,2,11,44,0.524,8,5,29,2,0.418,1,3,10,10,0,101,0.942,18,17.217,1.275,1.081,0.481,81.0,19.957
3,"Carr, Austin",28,24.0,0.287,94,10,27,4,1,1,17,36,0.383,9,4,33,1,0.374,0,1,0,0,0,108,0.757,21,12.583,1.044,0.710,0.433,67.5,12.583
4,"Newkirk, Sam",28,23.0,0.242,91,5,22,10,1,1,10,37,0.407,4,4,20,1,0.303,0,0,1,1,0,99,0.710,10,10.126,0.952,0.657,0.300,55.0,10.318
5,"Pawlik, Jeff",28,24.0,0.218,87,12,19,5,0,0,8,24,0.276,7,5,29,1,0.310,1,0,1,2,1,100,0.586,14,6.638,0.804,0.521,0.322,48.5,6.500


#### Crossroads League Statistics

Now that we have a few more advanced individual batting statistics, we need to gather the Crossroads League totals in order to incorporate them into our statistics and eventually rank the various hitters based on different stats.

To do this, we first subset each of the teams' batting table to extract only the Total row at the bottom. 

In [6]:
bat_totals = [df[df.Batting.str.contains("Total:", regex = False)] for df in dfb]
bat_totals[2]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
31,Total:,28,NaN,0.224,839,119,188,48,5,10,100,276,0.329,95,33,258,10,0.324,8,11,34,46,12,986,0.653,125,83.627,0.877,0.633,0.307,576.5,85.884


Next, we combine each of these rows into a new dataframe named `merged_bat_totals` using the code `pd.concat`. Below we show the first four rows of this new table.

In [7]:
merged_bat_totals = pd.concat(bat_totals)
merged_bat_totals[:4]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR
26,Total:,28,NaN,0.245,822,119,201,40,5,14,103,293,0.356,109,26,217,20,0.349,7,17,25,29,4,981,0.705,142,97.562,0.950,0.677,0.313,594.5,101.618
32,Total:,28,NaN,0.208,814,71,169,22,2,3,67,204,0.251,91,19,275,7,0.299,8,6,19,25,6,938,0.550,142,58.608,0.758,0.496,0.305,406.5,60.586
31,Total:,28,NaN,0.224,839,119,188,48,5,10,100,276,0.329,95,33,258,10,0.324,8,11,34,46,12,986,0.653,125,83.627,0.877,0.633,0.307,576.5,85.884
24,Total:,28,NaN,0.285,854,163,243,57,8,17,141,367,0.430,81,31,171,7,0.362,15,12,74,88,14,993,0.792,161,127.174,1.077,0.839,0.332,785.5,136.805


Now that we have our Totals table, we add a total row at the bottom using the code `.sum()`. Since the column labeled "Batting" does not provide any valuable information, we remove it from the table. Then, we isolate the totals row at the bottom with the code `.iloc[-1:]` to put the batting totals into a series.

In [8]:
merged_bat_totals.loc["CL_Total"] = merged_bat_totals.sum()
del merged_bat_totals["Batting"]
CL_bat_totals = merged_bat_totals.iloc[-1,:]

We make a copy to avoid any errors from overwriting the values, and then we calculate a few more statistics and run the `calcBatStats` function on the series. We printed out a few of the league totals.

In [9]:
CL_tot_b = CL_bat_totals.copy()
# CL On-Base Percentage
CL_tot_b["OBP"] = round((CL_tot_b["H"] + CL_tot_b["BB"] + CL_tot_b["HBP"]) / (CL_tot_b["AB"] + CL_tot_b["BB"] + CL_tot_b["HBP"] + CL_tot_b["SF"]), 3)
# CL Slugging Percentage
CL_tot_b["SLG"] = round(CL_tot_b["TB"] / CL_tot_b["AB"], 3)
# CL weighted On-Base Average (excluding IBB, formula from https://library.fangraphs.com/offense/woba/)
CL_tot_b["wOBA"] = round((0.69*CL_tot_b["BB"] + 0.72*CL_tot_b["HBP"] + 0.89*CL_tot_b["1B"] + 1.27*CL_tot_b["2B"] + 1.62*CL_tot_b["3B"] + 2.10*CL_tot_b["HR"]) / (CL_tot_b["AB"] + CL_tot_b["BB"] + CL_tot_b["SF"] + CL_tot_b["HBP"]), 3)
# CL Runs per Plate Appearance
CL_tot_b["R_per_PA"] = round(CL_tot_b["R"] / CL_tot_b["PA"], 3)
CL_tot_b["wRC_per_PA"] = CL_tot_b["R_per_PA"]
# ABF (statistic that incorporates the value of various methods of reaching base with regard to their assistance towards scoring runs)
CL_tot_b["ABF"] = round((0.47*CL_tot_b["1B"] + 0.38*CL_tot_b["2B"] + 0.55*CL_tot_b["3B"] + 0.93*CL_tot_b["HR"] + 0.33*CL_tot_b["BB"] + CL_tot_b["HBP"]) / (CL_tot_b["AB"] - CL_tot_b["H"]), 3)
# Calculate other totals such as AVG, OPS, POP, etc
calcBatStats(CL_tot_b)
# deleted Games Started column because it doesn't apply to league totals
del CL_tot_b["GS"]

In [10]:
CL_tot_b[:5]

GP      280.000
AVG       0.266
AB     8495.000
R      1548.000
H      2263.000
Name: CL_Total, dtype: float64

#### Advanced Statistics/Ranking Hitters

We would like to be able to rank hitters across the Crossroads League, and we will do this with a statistic called Weighted Runs Created Plus (wRC+). Below we calculate a few other statistics that will get us closer to our calculation of wRC+ for each hitter. 

In [11]:
for df in dfb:
    #OPS+
    df["OPS_plus"] = round(100 * ((df["OBP"]/CL_tot_b["OBP"]) + (df["SLG"]/CL_tot_b["SLG"]) - 1), 3)
    #Weighted On-Base Average
    df["wOBA"] = round(((0.69*df["BB"]) + (0.72*df["HBP"]) + (0.89*df["1B"]) + (1.27*df["2B"]) + (1.62*df["3B"]) + (2.1*df["HR"])) / (df["AB"] + df["BB"] + df["SF"] + df["HBP"]), 3)
    #Batting Runs
    df["Bat_Runs"] = round(((0.69*df["BB"]) + (0.72*df["HBP"]) + (0.89*df["1B"]) + (1.27*df["2B"]) + (1.62*df["3B"]) + (2.1*df["HR"])) / (df["AB"] + df["BB"] + df["SF"] + df["HBP"]), 3)
    #Weighted Runs Above Average (verify constant)
    df["wRAA"] = round(((df["wOBA"] - CL_tot_b["wOBA"]) / (1.157)) * df["PA"], 3)
    #Weighted Runs Created
    df["wRC"] = round(df["wRAA"] + (df["PA"] * (CL_tot_b["R"])/(CL_tot_b["PA"])), 3)
    #K/9
    df["K_pct"] = round(df["SO"]/df["AB"],2)

Since wRC+ incorporates the park factor for each hitter's home park, we have to load our various park factors for each team.

In [12]:
with open('Park_Factor.pkl', 'rb') as f:
    park_factor = pickle.load(f)

Now that we have our park factors, we can calculate our wRC+ for each hitter.

In [13]:
#Weighted Runs Created+
for i in range(len(teams)):
    dfb[i]["wRC_plus"] = round(100 * ((((dfb[i]["wRAA"] / dfb[i]["PA"]) + CL_tot_b["R_per_PA"]) + (CL_tot_b["R_per_PA"] - (park_factor[i]) * (CL_tot_b["R_per_PA"]))) / (CL_tot_b["wRC_per_PA"])), 3)

Before we begin ranking the hitters, we subset the data to only include hitters that have at least 27 at bats, which equates to one per conference game. 

In [14]:
#make sure everyone has at least 16 at bats (1 per game)
for i in range(len(teams)):
    dfb[i] = dfb[i][dfb[i]['AB'] >= 16]

We make copies of our dataframes to avoid errors and then we pretty the dataframes by adding a column for each player's team, removing the totals and opponents rows, rearranging the table's columns to show the important information first, and removing the unneccesary decimal in the `GS` column.

In [15]:
temp_dfb = [] 
for df in dfb: #make copy to avoid errors
    temp_dfb.append(df.copy())
for i in range(len(teams)): #add column for team
    temp_dfb[i]["Team"] = teams[i]
for df in temp_dfb: 
    df.drop(df.tail(2).index,inplace=True) # drop last 2 rows (only run this line once or data will be lost)
    team = df.pop("Team")
    df.insert(1, team.name, team) #move team column to second
    k = df.pop("K_pct")
    df.insert(16, k.name, k) #move team column to second
    stat = df.pop("wRC_plus")
    df.insert(4, stat.name, stat) #move wRC+ column to front of statistical columns
    df['GS'] = df['GS'].astype(int) #remove decimal place on GS column
temp_dfb[2][-2:] #verify totals and opponents rows are gone

,Batting,Team,GP,GS,wRC_plus,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,K_pct,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR,OPS_plus,wOBA,Bat_Runs,wRAA,wRC
11,"Turner, Dalton",Grace,12,11,45.057,0.125,32,3,4,3,0,0,4,7,0.219,4,1,0.28,9,0,0.243,0,0,2,3,1,37,0.462,1,1.556,0.587,0.483,0.174,18.0,1.589,22.756,0.221,0.221,-3.774,1.938
15,"Buschman, Grant",Grace,16,11,9.145,0.053,38,2,2,1,0,0,4,3,0.079,5,2,0.53,20,2,0.200,0,1,0,0,0,46,0.279,1,0.488,0.332,0.256,0.111,11.5,0.488,-24.598,0.157,0.157,-7.236,-0.134


With our more readable data, we combine each team's table into one dataframe wiht the code `pd.concat`. Then, we sort the table by wRC+ to show the hitters with the highest wRC+ first. Finally, we display our top ten hitters.

In [20]:
all_hitters = pd.concat(temp_dfb) #collect all in one dataframe
top_hitters = all_hitters.sort_values(by=['wRC_plus'], ascending=False) #sort by wRC+ in descending order
top_hitters[:10] #top 10 hitters

,Batting,Team,GP,GS,wRC_plus,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,K_pct,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR,OPS_plus,wOBA,Bat_Runs,wRAA,wRC
1,"Wilson, Satchell",HU,28,28,193.025,0.396,96,22,38,6,2,3,21,57,0.594,11,4,0.09,9,1,0.469,2,1,15,18,3,114,1.063,27,26.103,1.459,1.338,0.407,120.5,28.227,180.297,0.457,0.457,11.627,29.227
2,"Saal, Aaron",MVNU,28,28,185.002,0.368,95,25,35,8,1,6,27,63,0.663,14,1,0.12,11,1,0.442,3,0,12,13,1,113,1.105,20,28.321,1.473,1.385,0.358,134.0,31.156,190.286,0.465,0.465,12.306,29.751
4,"Thixton, Tye",IWU,28,28,179.485,0.393,117,39,46,8,1,8,30,80,0.684,9,3,0.12,14,3,0.450,0,1,11,11,0,130,1.134,29,34.921,1.527,1.373,0.400,164.5,38.282,197.818,0.487,0.487,16.629,36.699
6,"Blinn, Denver",IWU,28,28,178.927,0.364,107,39,39,15,3,5,21,75,0.701,11,7,0.15,16,0,0.452,1,0,12,13,1,126,1.153,16,31.780,1.517,1.500,0.391,152.5,34.341,202.676,0.486,0.486,16.009,35.461
2,"Porcellato, Matteo",Marian,28,28,176.735,0.367,98,23,36,9,1,1,21,50,0.510,14,9,0.15,15,1,0.484,1,1,7,9,2,123,0.994,25,22.321,1.361,1.194,0.422,108.0,23.116,163.186,0.439,0.439,10.631,29.620
3,"Glover, Jake",MVNU,28,28,169.285,0.326,95,14,31,7,0,6,30,56,0.589,15,4,0.12,11,1,0.431,2,0,0,1,1,116,1.020,18,23.418,1.346,1.103,0.312,107.5,22.703,168.505,0.437,0.437,9.825,27.733
7,"Jenkins, Colby",IWU,28,28,162.088,0.351,97,36,34,7,2,5,39,60,0.619,17,3,0.21,20,1,0.454,2,2,8,10,2,121,1.073,20,26.842,1.424,1.257,0.392,151.5,27.710,182.471,0.456,0.456,12.236,30.916
4,"Jones, Caden",Marian,28,28,161.583,0.327,98,22,32,4,0,5,18,51,0.520,8,7,0.33,32,0,0.416,0,0,0,0,0,113,0.936,23,19.245,1.263,1.000,0.443,95.0,19.245,146.881,0.412,0.412,7.130,24.575
1,"Griffin, Chris",Grace,28,24,158.995,0.344,96,17,33,8,0,4,14,53,0.552,15,1,0.20,19,0,0.430,2,0,2,2,0,114,0.982,21,22.919,1.326,1.092,0.387,93.5,23.524,158.860,0.424,0.424,8.375,25.975
4,"Dice, Camden",HU,28,28,158.225,0.326,86,13,28,9,0,3,20,46,0.535,6,3,0.28,24,0,0.381,2,1,1,1,0,98,0.916,16,17.000,1.242,0.918,0.410,83.0,17.259,140.983,0.395,0.395,4.743,19.873


In [17]:
dfb[2]

,Batting,GP,GS,AVG,AB,R,H,2B,3B,HR,RBI,TB,SLG,BB,HBP,SO,GDP,OBP,SF,SH,SB,SBA,CS,PA,OPS,1B,RC,POP,TA,BABIP,HEQO,BSR,OPS_plus,wOBA,Bat_Runs,wRAA,wRC,K_pct,wRC_plus
1,"Griffin, Chris",28,24.0,0.344,96,17,33,8,0,4,14,53,0.552,15,1,19,0,0.430,2,0,2,2,0,114,0.982,21,22.919,1.326,1.092,0.387,93.5,23.524,158.860,0.424,0.424,8.375,25.975,0.20,158.995
2,"Danforth, Patrick",28,24.0,0.333,84,12,28,6,2,2,11,44,0.524,8,5,29,2,0.418,1,3,10,10,0,101,0.942,18,17.217,1.275,1.081,0.481,81.0,19.957,148.448,0.410,0.410,6.198,21.791,0.35,151.139
3,"Carr, Austin",28,24.0,0.287,94,10,27,4,1,1,17,36,0.383,9,4,33,1,0.374,0,1,0,0,0,108,0.757,21,12.583,1.044,0.710,0.433,67.5,12.583,100.563,0.342,0.342,0.280,16.953,0.35,112.974
4,"Newkirk, Sam",28,23.0,0.242,91,5,22,10,1,1,10,37,0.407,4,4,20,1,0.303,0,0,1,1,0,99,0.710,10,10.126,0.952,0.657,0.300,55.0,10.318,86.971,0.313,0.313,-2.225,13.059,0.22,96.696
5,"Pawlik, Jeff",28,24.0,0.218,87,12,19,5,0,0,8,24,0.276,7,5,29,1,0.310,1,0,1,2,1,100,0.586,14,6.638,0.804,0.521,0.322,48.5,6.500,55.746,0.272,0.272,-5.791,9.647,0.33,73.686
6,"Rich, Alex",28,24.0,0.206,102,11,21,5,0,1,12,29,0.284,7,5,15,1,0.287,1,0,6,9,3,115,0.571,15,7.450,0.777,0.547,0.230,61.5,7.411,51.400,0.263,0.263,-7.554,10.200,0.15,68.636
7,"Snyder, R.J.",22,12.0,0.204,49,10,10,0,0,0,2,10,0.204,0,0,10,1,0.204,0,3,6,11,5,52,0.408,10,2.041,0.612,0.333,0.256,28.0,1.315,8.155,0.182,0.182,-7.056,0.972,0.20,23.178
8,"Haney, Houston",23,16.0,0.177,62,6,11,3,1,1,6,19,0.306,5,3,17,1,0.264,2,1,0,0,0,73,0.570,6,4.537,0.747,0.491,0.217,33.5,4.537,50.599,0.257,0.257,-5.174,6.096,0.27,65.266
10,"Mullet, Jaron",26,16.0,0.132,53,12,7,2,0,0,6,9,0.170,17,1,29,0,0.347,1,1,3,4,1,73,0.517,5,3.086,0.649,0.612,0.280,38.5,3.596,39.160,0.270,0.270,-4.354,6.916,0.55,72.561
11,"Turner, Dalton",12,11.0,0.125,32,3,4,3,0,0,4,7,0.219,4,1,9,0,0.243,0,0,2,3,1,37,0.462,1,1.556,0.587,0.483,0.174,18.0,1.589,22.756,0.221,0.221,-3.774,1.938,0.28,45.057
